# Code Explanation

This script implements the **White Test for Heteroscedasticity**, a statistical procedure to assess whether the variance of residuals in a regression model is constant (homoscedasticity). The test checks for relationships between residual variance and the independent variables, their squares, and their cross-products.

## `white_test` Function

### 1. **Inputs**
- **`data`**: Dataset where the first column is the dependent variable (\(Y\)), and the remaining columns are independent variables.
- **`model_results`**: Regression output (e.g., coefficients and residuals) from a function like `linear_model` or `mle_model`.
- **`feature_names`** *(optional)*: List of variable names, including the dependent variable.

### 2. **Steps to Perform the White Test**

#### **Data Preparation**
- Converts the input to a numpy array if provided as a pandas DataFrame.
- Constructs the design matrix (\(X\)) with an intercept term.

#### **Residual Calculation**
- Uses the regression coefficients from `model_results` to compute predicted values.
- Calculates residuals as \(Y - X\beta\) and squares them (\(e^2\)) for the test.

#### **White Test Regressors**
- Constructs auxiliary regressors:
  - Squares of independent variables.
  - Cross-products of independent variables (combinations with replacement).
- Combines these with the original regressors to form a new design matrix (\(X_{\text{White}}\)).

#### **Auxiliary Regression**
- Fits a regression model where the dependent variable is the squared residuals (\(e^2\)), and the independent variables are the regressors from \(X_{\text{White}}\).
- Calculates the test statistic:
  \[
  W = n \cdot R^2
  \]
  - \(n\): Number of observations.
  - \(R^2\): Coefficient of determination from the auxiliary regression.

#### **Statistical Inference**
- Degrees of freedom (\(df\)) are equal to the number of regressors in the auxiliary model.
- Calculates the \(p\)-value using the chi-squared distribution:
  \[
  p = 1 - \text{CDF}_{\chi^2}(W, df)
  \]
- Determines whether to reject the null hypothesis of homoscedasticity at a significance level (\(\alpha = 0.05\)).

### 3. **Output**
- Returns a dictionary containing:
  - **Summary**: Text-based explanation of the test results.
  - **White Test Statistic**: Value of \(W\).
  - **Degrees of Freedom**: Number of regressors in the auxiliary regression.
  - **P-value**: Probability of observing the test statistic under the null hypothesis.
  - **Conclusion**: Whether homoscedasticity is rejected.
  - **Auxiliary \(R^2\)**: Goodness-of-fit from the auxiliary regression.
  - **Auxiliary Model**: Statsmodels object of the auxiliary regression.

### 4. **Error Handling**
- Catches linear algebra errors (e.g., singular matrix) that may occur due to perfect multicollinearity in the auxiliary regression.
- Provides a descriptive message when the test cannot be performed.

### 5. **Use Case**
The White Test is a diagnostic tool to check for heteroscedasticity in regression models. It is especially useful in cases where residual variance might depend on the independent variables or their combinations.


In [ ]:
import numpy as np
from scipy import stats, optimize
from tabulate import tabulate
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def mle_model(data, feature_names=None):
    """
    Maximum Likelihood Estimation (MLE) for Linear Model
    """
    # Data preparation
    if isinstance(data, pd.DataFrame):
        if feature_names is None:
            feature_names = data.columns.tolist()
        data = data.values
    
    if feature_names is None:
        feature_names = ['Y'] + [f'X{i}' for i in range(1, data.shape[1])]
    
    if data.shape[1] < 2:
        raise ValueError("The data must contain at least two columns")
    
    # Extract variables
    y = data[:, 0].reshape(-1, 1)
    X = np.column_stack([np.ones(len(data)), data[:, 1:]])
    
    def log_likelihood(params):
        """Negative log-likelihood function"""
        beta = params[:-1].reshape(-1, 1)
        sigma = np.exp(params[-1])  # Ensure sigma is positive
        residuals = y - X @ beta
        ll = -0.5 * len(X) * np.log(2 * np.pi * sigma**2) - \
             (1 / (2 * sigma**2)) * np.sum(residuals**2)
        return -ll  # Return negative because we're minimizing
    
    # Initialize parameters (beta and log(sigma))
    init_params = np.zeros(X.shape[1] + 1)
    init_params[-1] = np.log(np.std(y))
    
    # Optimize using MLE
    mle_result = optimize.minimize(
        log_likelihood,
        init_params,
        method='BFGS',
        options={'disp': False}
    )
    
    # Extract results
    beta_mle = mle_result.x[:-1].reshape(-1, 1)
    sigma_mle = np.exp(mle_result.x[-1])
    log_lik = -mle_result.fun
    
    # Calculate standard errors using OLS approach
    y_hat = X @ beta_mle
    residuals = y - y_hat
    n = len(X)
    p = X.shape[1]
    
    # Calculate MSE
    mse = np.sum(residuals**2) / (n - p)
    
    # Calculate standard errors
    try:
        XtX_inv = np.linalg.inv(X.T @ X)
        se_beta_mle = np.sqrt(np.diag(XtX_inv * mse))
        
        # Calculate test statistics
        t_values = beta_mle.flatten() / se_beta_mle
        p_values = 2 * stats.t.sf(np.abs(t_values), df=n-p)
    except np.linalg.LinAlgError:
        print("Warning: Could not compute standard errors (singular matrix)")
        se_beta_mle = np.full(len(beta_mle), np.nan)
        t_values = np.full(len(beta_mle), np.nan)
        p_values = np.full(len(beta_mle), np.nan)
    
    # Calculate R-squared statistics
    SS_tot = np.sum((y - np.mean(y))**2)
    SS_res = np.sum(residuals**2)
    r_squared = 1 - (SS_res / SS_tot)
    adj_r_squared = 1 - (1 - r_squared) * ((n - 1) / (n - p))
    
    # Calculate AIC and BIC
    k = len(init_params)  # number of parameters (including sigma)
    aic = 2 * k - 2 * log_lik
    bic = np.log(n) * k - 2 * log_lik
    
    # Create diagnostic plots
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # Residuals vs Fitted
    axes[0,0].scatter(y_hat.flatten(), residuals.flatten())
    axes[0,0].axhline(y=0, color='r', linestyle='--')
    axes[0,0].set_xlabel('Fitted values')
    axes[0,0].set_ylabel('Residuals')
    axes[0,0].set_title('Residuals vs Fitted')
    
    # Q-Q plot
    stats.probplot(residuals.flatten(), dist="norm", plot=axes[0,1])
    axes[0,1].set_title('Normal Q-Q Plot')
    
    # Scale-Location plot
    axes[1,0].scatter(y_hat.flatten(), np.sqrt(np.abs(residuals.flatten())))
    axes[1,0].set_xlabel('Fitted values')
    axes[1,0].set_ylabel('√|Residuals|')
    axes[1,0].set_title('Scale-Location Plot')
    
    # Density plot of residuals
    sns.kdeplot(data=residuals.flatten(), ax=axes[1,1])
    axes[1,1].set_title('Residuals Density Plot')
    
    plt.tight_layout()
    
    # Create coefficient table
    coef_data = []
    var_names = ['Intercept'] + feature_names[1:]
    for i in range(len(beta_mle)):
        coef_data.append([
            var_names[i],
            f"{beta_mle[i][0]:.4f}",
            f"{se_beta_mle[i]:.4f}",
            f"{t_values[i]:.4f}",
            f"{p_values[i]:.4f}"
        ])
    
    return {
        'summary': f"""
Maximum Likelihood Estimation Results
===================================
Dependent Variable: {feature_names[0]}
Number of Observations: {len(X)}
Number of Predictors: {X.shape[1]-1}

Model Statistics:
----------------
R-squared: {r_squared:.4f}
Adjusted R-squared: {adj_r_squared:.4f}
Log-likelihood: {log_lik:.4f}
AIC: {aic:.4f}
BIC: {bic:.4f}
Sigma (MLE): {sigma_mle:.4f}

Coefficients:
------------
{tabulate(coef_data, headers=['Variable', 'Coefficient', 'Std. Error', 't-value', 'p-value'], 
          tablefmt='pipe', floatfmt='.4f')}
""",
        'coefficients': pd.DataFrame(coef_data, 
                                   columns=['Variable', 'Coefficient', 'Std. Error', 't-value', 'p-value']),
        'r_squared': r_squared,
        'adj_r_squared': adj_r_squared,
        'log_likelihood': log_lik,
        'aic': aic,
        'bic': bic,
        'sigma': sigma_mle,
        'residuals': residuals.flatten(),
        'fitted_values': y_hat.flatten(),
        'diagnostic_plots': fig
    }



In [ ]:
import numpy as np
from scipy import stats
from itertools import combinations_with_replacement
import statsmodels.api as sm

def white_test(data, model_results, feature_names=None):
    """
    White Test for Heteroscedasticity
    
    Parameters:
    -----------
    data : numpy.ndarray or pandas.DataFrame
        Data where the first column is the dependent variable (Y) and 
        the remaining columns are the independent variables
    model_results : dict
        Results from the mle_model or linear_model function
    feature_names : list, optional
        Names of the features (including dependent variable)
        
    Returns:
    --------
    dict
        Dictionary containing the White test statistic, p-value, and conclusion
    """
    # Data preparation
    if isinstance(data, pd.DataFrame):
        if feature_names is None:
            feature_names = data.columns.tolist()
        data = data.values
    
    if feature_names is None:
        feature_names = ['Y'] + [f'X{i}' for i in range(1, data.shape[1])]
    
    # Extract variables
    y = data[:, 0].reshape(-1, 1)
    X = np.column_stack([np.ones(len(data)), data[:, 1:]])
    
    # Extract estimated coefficients and calculate residuals
    beta = model_results['coefficients']['Coefficient'].values
    beta = np.array([float(b) for b in beta]).reshape(-1, 1)
    residuals = y - X @ beta
    
    # Calculate squared residuals
    residuals_squared = residuals**2
    
    # Construct White test regressors
    X_no_constant = X[:, 1:]  # Remove constant term
    n, k = X_no_constant.shape
    
    # Create squares and cross-products
    squares = X_no_constant**2
    
    # Create cross-products
    cross_products = []
    for i, j in combinations_with_replacement(range(k), 2):
        if i != j:
            cross_products.append(X_no_constant[:, i] * X_no_constant[:, j])
    
    if cross_products:
        cross_products = np.column_stack(cross_products)
        X_white = np.column_stack([X, squares, cross_products])
    else:
        X_white = np.column_stack([X, squares])
    
    # Fit auxiliary regression
    try:
        auxiliary_model = sm.OLS(residuals_squared, X_white).fit()
        
        # Calculate White test statistic
        white_statistic = auxiliary_model.nobs * auxiliary_model.rsquared
        df = auxiliary_model.df_model  # Degrees of freedom
        white_pvalue = stats.chi2.sf(white_statistic, df)
        
        # Create conclusion
        alpha = 0.05  # significance level
        conclusion = ("Reject null hypothesis of homoscedasticity" 
                     if white_pvalue < alpha 
                     else "Fail to reject null hypothesis of homoscedasticity")
        
        # Create detailed results
        results = {
            'summary': f"""
White Test for Heteroscedasticity
================================
H0: Homoscedasticity (constant variance)
H1: Heteroscedasticity (non-constant variance)

Test Results:
------------
White test statistic: {white_statistic:.4f}
Degrees of freedom: {df}
P-value: {white_pvalue:.4f}

Conclusion (α = {alpha}):
{conclusion}
""",
            'white_statistic': white_statistic,
            'degrees_of_freedom': df,
            'p_value': white_pvalue,
            'conclusion': conclusion,
            'auxiliary_r_squared': auxiliary_model.rsquared,
            'auxiliary_model': auxiliary_model
        }
        
    except np.linalg.LinAlgError:
        results = {
            'summary': """
White Test for Heteroscedasticity
================================
Error: Singular matrix encountered. Cannot perform White test.
This might be due to perfect multicollinearity in the auxiliary regression.
""",
            'white_statistic': None,
            'degrees_of_freedom': None,
            'p_value': None,
            'conclusion': "Test could not be performed due to computational issues",
            'auxiliary_r_squared': None,
            'auxiliary_model': None
        }
    
    return results

